## LightGBM Hyperparameter Tuning (F1-Score)

In [19]:
# Load Libraries

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score

from lightgbm import LGBMClassifier

import optuna

In [21]:
# Load Data

df = pd.read_csv('bank_4.csv', index_col=0)

In [23]:
# Train / Test Split

X = df.drop(columns=['churn', 'complain', 'umap_1', 'umap_2'])
y = df['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [25]:
# Hyperparameter tuning

def objective(trial):
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', -1, 50),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 40),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0)
    }

    model = LGBMClassifier(
        **params,
        objective='binary',
        verbose=-1,
        random_state=42,
        n_jobs=-1)
    
    threshold = 0.34
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    scores = []
    
    for tr, te in skf.split(X_train, y_train):
        
        X_tr, X_te = X_train.iloc[tr], X_train.iloc[te]
        y_tr, y_te = y_train.iloc[tr], y_train.iloc[te]
        
        model.fit(X_tr, y_tr)
        prob = model.predict_proba(X_te)[:, 1]
        y_pred = np.where(prob < threshold, 0, 1)
        
        scores.append(f1_score(y_te, y_pred))
        
    return np.mean(scores)

In [29]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, show_progress_bar=True)    # More trials because LGB is faster

[I 2024-07-10 14:21:55,096] A new study created in memory with name: no-name-de09bcef-4ffe-40cb-b60a-7e041ca56d81


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2024-07-10 14:21:55,775] Trial 0 finished with value: 0.5912494527756585 and parameters: {'n_estimators': 179, 'max_depth': 25, 'learning_rate': 0.15525399369993084, 'num_leaves': 31, 'colsample_bytree': 0.8012711844106865, 'subsample': 0.7336885301818519}. Best is trial 0 with value: 0.5912494527756585.
[I 2024-07-10 14:21:58,090] Trial 1 finished with value: 0.6116050291502633 and parameters: {'n_estimators': 375, 'max_depth': 29, 'learning_rate': 0.04272767046260066, 'num_leaves': 26, 'colsample_bytree': 0.9764291183902076, 'subsample': 0.7662198214992367}. Best is trial 1 with value: 0.6116050291502633.
[I 2024-07-10 14:21:59,516] Trial 2 finished with value: 0.5769090314587457 and parameters: {'n_estimators': 400, 'max_depth': 42, 'learning_rate': 0.27558455769382456, 'num_leaves': 25, 'colsample_bytree': 0.6532547571657714, 'subsample': 0.6464286393408176}. Best is trial 1 with value: 0.6116050291502633.
[I 2024-07-10 14:22:02,706] Trial 3 finished with value: 0.56953872595120

In [31]:
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)

Best trial: FrozenTrial(number=98, state=1, values=[0.6235876372607708], datetime_start=datetime.datetime(2024, 7, 10, 14, 25, 15, 505634), datetime_complete=datetime.datetime(2024, 7, 10, 14, 25, 16, 468896), params={'n_estimators': 191, 'max_depth': 19, 'learning_rate': 0.025294173033637454, 'num_leaves': 30, 'colsample_bytree': 0.7501200962447905, 'subsample': 0.5239120285973756}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=1000, log=False, low=100, step=1), 'max_depth': IntDistribution(high=50, log=False, low=-1, step=1), 'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'num_leaves': IntDistribution(high=40, log=False, low=20, step=1), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None)}, trial_id=98, value=None)
Best hyperparameters: {'n_estimators': 191, 'max_depth': 19, 'learning_rate': 0.